# Creates "Contextualized" Entity Embeddings

We will use the same requirements as in the entity_embedding_tutorial.ipynb file.

The unique entity embeddings, before contextualization, that are used in Bootleg's neural model are not as useful in downstream tasks in isolation. They were not trained to have a meaningful dot product, for example. Instead, we create "contextualized" entity embeddings that a fed through the Bootleg model.

Bootleg requires two things generating a "contextualized" entity embeddings:
1. A sentence
2. A mention and list of entity candidates

We want to generate (1) and (2) that represents, as closely as possible, a single entity. Our solution is
1. The sentence is the entity title and nothing else
2. The mention is the entiyt title and there is a single candidate. The candidate is the entity we want to generate an embedding for.

We do this below to create embeddings.

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [10]:
import sys, torch
from rich import print
from rich.progress import track
import numpy as np
from pathlib import Path

## Loading

In [3]:
from bootleg.end2end.bootleg_annotator import BootlegAnnotator
bootleg_cache = "tutorial_data" # WHERE DATA IS DOWNLOADED
device = -1 # SET TO 0 FOR GPU
ann = BootlegAnnotator(cache_dir=bootleg_cache, device=device, return_embs=True, verbose=True)

[2021-05-02 20:31:39,897][INFO] emmental.meta:119 - Setting logging directory to: /dfs/scratch0/lorr1/projects/bootleg/tutorial_data/data/log_dir/2021_05_02/20_31_39/d0ebc1e1
[2021-05-02 20:31:39,962][INFO] emmental.meta:62 - Loading Emmental default config from /dfs/scratch0/lorr1/env_bootleg_38/lib/python3.8/site-packages/emmental/emmental-default-config.yaml.
[2021-05-02 20:31:39,963][INFO] emmental.meta:171 - Updating Emmental config from user provided config.
[2021-05-02 20:31:39,965][INFO] emmental.utils.seed:27 - Set random seed to 1234.
[2021-05-02 20:34:27,761][INFO] emmental.model:72 - Created emmental model Bootleg that contains task set().
[2021-05-02 20:34:31,256][INFO] bootleg.tasks.task_getters:12 - Loading embeddings...
[2021-05-02 20:38:07,497][INFO] emmental.task:78 - Created task: NED
[2021-05-02 20:38:07,500][INFO] emmental.model:108 - Moving bert module to CPU.
[2021-05-02 20:38:07,505][INFO] emmental.model:108 - Moving embedding_payload module to CPU.
[2021-05-02 

In [4]:
from bootleg.utils.utils import get_lnrm

def get_mention(ep, qid):
    """Returns the lower cased title with punction stripped"""
    return get_lnrm(ep.get_title(qid), strip=True, lower=True)

In [6]:
from bootleg.symbols.entity_profile import EntityProfile

ep = EntityProfile.load_from_cache(Path(bootleg_cache) / "data/entity_db")

## Generate embeddings

To show how the process works, we'll use a small subset of entities, create embeddings for them, and compare their cosine similarity. The entity set we'll use is a collection of all entities with the word vmware or apple in their title.

In [11]:
# Collect entities
entities_to_emb = [q for q in track(ep.get_all_qids()) if ("vmware" in ep.get_title(q).lower()) or ("apple" in ep.get_title(q).lower())]

print([ep.get_title(q) for q in entities_to_emb])

Output()

[
    'Appleton P. Clark Jr.',
    'You Are the Apple of My Eye',
    'The Forgotten (Applegate novel)',
    'Katy (apple)',
    'The Apple Dumpling Gang (film)',
    'John Appleton',
    'Cairnpapple Hill',
    'Daniel Appleton',
    'Mark Appleyard',
    'Golden apple',
    'Appleton (surname)',
    'Eva (apple)',
    'David Applebee',
    'Migration Assistant (Apple)',
    'Major Applewhite',
    'Institute of Physics Edward Appleton Medal and Prize',
    'AppleWorks User Group',
    'The Apple Dumpling Gang',
    'James Cappleman',
    'The Resistance (Applegate novel)',
    'Brushy Mountain Apple Festival',
    'Apple Beer',
    'Maiden Blush apple',
    'Beneath Apple Manor',
    'JJ Appleton',
    'Big Apple Academy',
    'Sid Applebaum',
    'Applegate Lake',
    'Apple Shampoo',
    'Betty Applewhite',
    'John Appleton Wilson',
    'Apples and Bananas',
    'Apple A12X',
    "Appletons' Cyclopædia of American Biography",
    'Michael Appleby (politician)',
    'Surprise (apple)',
    'Sappleton',
    'Champion (apple)',
    'Scrapple from the Apple',
    'Diced Pineapples',
    'Pat Apple',
    'Golden Apples of the Sun (film)',
    '1905 Appleby by-election',
    'Reloaded (Green Apple Quick Step album)',
    'Kentucky Apple Festival',
    'The Apple Cart',
    'Appletreewick',
    'Apple (band)',
    'Wasp Star (Apple Venus Volume 2)',
    'Fred Applegate',
    'Applewood Park, Calgary',
    'Timeline of Apple Inc. products',
    'Appleton Technical Academy',
    'Appleby Hall',
    'Apple Productivity Experience Group',
    'Appleton Milo Harmon',
    'Measure (Apple)',
    'Flamenco (apple)',
    'Zapple Monitor',
    'Red Applegate',
    'Grapple (tool)',
    "Adam's Apple (horse)",
    'Family Red Apple boycott',
    'Enterprise (apple)',
    'Motorola Mobility v. Apple Inc.',
    'Calendar (Apple)',
    'Apple Open Directory',
    'Develop (Apple magazine)',
    'Hot Apple Pie',
    'Samuel Appleton (merchant)',
    'HMS Grappler (1856)',
    'Apple ProDOS',
    'Robert Appleby Bartram (shipbuilder)',
    'Belinda Chapple',
    'Francis Appleton Harding',
    'Christopher Apple House',
    'Pristine apple',
    'Appleby railway station (Lincolnshire)',
    'Len Appleton',
    'Kendra Appleton',
    'Alison Appleton',
    'List of Apple II clones',
    'VMware vSphere',
    'Astragalus applegatei',
    'List of awards and nominations received by Fiona Apple',
    'VMware Server',
    'Pineapple pit',
    'William M. Appleton',
    'Richard Appleton',
    'Apple Developer',
    'Appledore railway station (Devon)',
    'Appledore, Kent',
    'Appleton, Maine',
    'Peter Chapple-Hyam',
    'List of Apple II games',
    'Applegate–Fairbairn fighting knife',
    'Baki the Grappler',
    'AppleShare IP Migration',
    'Apple Display Connector',
    'Apple Jack',
    'Apple Miyuki',
    'Bicolored apple',
    'George Chapple Norton',
    'The Stranger (Applegate novel)',
    'Appley Bridge meteorite',
    'Apple ID',
    'Appleseed Ex Machina',
    'William E. Applegate',
    'Matthew Appleyard (died 1700)',
    'Appleby Frodingham F.C.',
    'Pork chops and applesauce',
    'Fantasy (Appleton song)',
    'Quba Apple Festival',
    'Typography of Apple Inc.',
    'Andrew Appleton',
    'Apple–Intel architecture',
    'Michael Appleton',
    'Apple Bottoms',
    'Apple Color Emoji',
    'Bungo Stray Dogs: Dead Apple',
    'Appleton, Illinois',
    'Appleby, New Zealand',
    'Sugar-apple',
    'Apple Configurator',
    'Apple IIc Plus',
    'Blue Meanies (Apple Computer)',
    'Apple Grove, Mason County, West Virginia',
    'Apple Specialist',
    'Eupithecia grappleri',
    'Criticism of Apple Inc.',
    'Apple Comics',
    'Apple Multiple Scan 14 Display',
    'Apple Battery Charger',
    'Grenadier (apple)',
    'Contacts (Apple)',
    'Appleton International Airport',
    'Duchess of Oldenburg (apple)',
    'Eliot Chapple',
    'Apple Tree Creek, Queensland',
    'Appletee',
    'The Candy Apple News Company',
  

In [13]:
# Create the input of (1) and (2) to feed into our annotator
extracted_exs = [
    {
        "sentence": ep.get_title(q),
        "aliases": [get_mention(ep, q)],
        "spans": [[0, len(ep.get_title(q).split())]],
        "cands": [[q]],
    }
    for q in entities_to_emb
]
# We use the special `extracted_examples` input into our label_mentions so that annotator uses the candidates and sentence we provide
out_dict = ann.label_mentions(extracted_examples=extracted_exs)

In [14]:
# Get the ids of the VMWare entities only
vmware_ids = [i for i, t in enumerate([ep.get_title(q) for q in entities_to_emb]) if "vmware" in t.lower()]
print(len(vmware_ids), len(entities_to_emb))

13 1740

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

mat = np.vstack(out_dict["embs"])
print(mat.shape)
res = cosine_similarity(mat)
sorted_res = np.argsort(res, axis=-1)
print(sorted_res.shape)
# For each vmware entity, get the top 5 most similar entities from the set above
for i in vmware_ids:
    for j in sorted_res[i][::-1][:5]:
        print(ep.get_title(entities_to_emb[i]), "~", ep.get_title(entities_to_emb[j]))

(1740, 512)

(1740, 1740)

VMware vSphere ~ VMware vSphere

VMware vSphere ~ VMware Horizon

VMware vSphere ~ VMware ESXi

...

VMware VMFS ~ VMware VMFS

VMware VMFS ~ VMware Infrastructure

VMware VMFS ~ VMware ThinApp

VMware VMFS ~ VMware Workstation Player

VMware VMFS ~ VMware Server

We see that overall, we are choosing similar entities. No people with the name of Apple, for example, are returned as being similar.